## 0 | Import packages

In [1]:
import os
import h5py
import numpy as np
import pandas as pd
import tkinter
from tkinter.filedialog import askopenfilename, askopenfilenames
from collections import defaultdict
from nptdms import TdmsFile
print("done!")

done!


## 1 | Open an .hdf5 file and extract channels

In [2]:
# This was Yu Lin's function:
def openHDF5file(in_path, channel_list = ['Channel A', 'Channel B', 'Output A', 'Output B'], curated_channel = None):
    hdf5_file = h5py.File(in_path, 'r')
     
    dataDict = defaultdict(list)
     
    for channel in hdf5_file.keys():
        
        #to fix hdf5 indexing
        if 'Channel' in channel or 'Output' in channel:
            trialKeysInHDF5 = list(hdf5_file[channel].keys()) # which has been sorted alphabatically as in strings. can be simulated by sorted([str(x) for x in range(numTrials)]) 
            sortI, numericalOrder = zip(*sorted(enumerate([int(x) for x in trialKeysInHDF5]),key=lambda i:i[1])) # resort numerically, and get the sorting indexes 
            
            if curated_channel is not None: 
                sortI, numericalOrder = zip(*sorted(enumerate([int(x) for x in list(hdf5_file[curated_channel].keys())]),
                                                    key=lambda i:i[1]))
                if 'Channel' in channel: trialKeysInHDF5 = list(hdf5_file[curated_channel].keys())
                if 'Output' in channel: sortI = [int(x)-1 for x in numericalOrder] #note, may not work as expected if very first trial is deleted ? but doesnt matter as output is usually the same across trials in a given file
#            print(channel, trialKeysInHDF5, sortI)
        else:
            trialKeysInHDF5 = list(hdf5_file[channel].keys())
            sortI = range(len(trialKeysInHDF5))
#            print('not sorted')
            
        for i in range(len(sortI)):
            correctedTrialKey = trialKeysInHDF5[sortI[i]] 
            dataDict[channel].append(np.array(hdf5_file[channel][correctedTrialKey]))
   
    extractedChannels = []
    
    for channel in channel_list:
        extractedChannels.append(dataDict[channel])

    
    if len(dataDict['Time']) >0:
        time = dataDict['Time'][0]
        dt = np.mean(np.diff(time))
    else:
        dt = 0.04
        time = np.linspace(0, len(dataDict['Channel A'][0])*dt, len(['Channel A'][0]))
    
    return extractedChannels, time, dt

### 1.1 | Load an .hdf5 file

In [3]:
# Choose a .hdf5 file to test the code
root = tkinter.Tk()
root.attributes('-topmost', True) # Make window appear on top
in_path = askopenfilename() # Open dialogue to select file
print(in_path)
root.destroy() # Close the root window

# Split the pathname path into a pair, (head, tail) where tail is the last pathname component and head is everything leading up to that.
folder_name = os.path.split(in_path)[0] # Get path until folder
file_name = os.path.split(in_path)[1] # Get filename
print(folder_name)
print(file_name)

D:/Dropbox (UCL)/Project_paginhibition/analysis/loose_seal/analysis_sample_cells/dlpag_vgat_171114_c4_LIAM_OP_clear_VC_2.hdf5
D:/Dropbox (UCL)/Project_paginhibition/analysis/loose_seal/analysis_sample_cells
dlpag_vgat_171114_c4_LIAM_OP_clear_VC_2.hdf5


### 1.2 | Check how zip() and sorted() work

In [4]:
#print(in_path)
test_file = h5py.File(in_path, 'r')
print(list(test_file.keys()))
test_file['Channel B'].keys()

trialKeysInHDF5 = list(test_file['Channel B'].keys())
print(trialKeysInHDF5)

# This is how Yu Lin was doing the sorting:
dataDict = defaultdict(list)
sortI, numericalOrder = zip(*sorted(enumerate([int(x) for x in trialKeysInHDF5]), 
                                    key = lambda i:i[1]))

print(sortI)
print(numericalOrder)

['Channel A', 'Channel B', 'Output A', 'Output B', 'Sweeps_Analysis', 'Time']
['1', '10', '100', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '4', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '5', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '6', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '7', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '8', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '9', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99']
(0, 12, 23, 34, 45, 56, 67, 78, 89, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 79, 80, 81, 82, 83, 8

In [5]:
# How zip() and sorted() work:
a = ("John", "Charles", "Mike")
b = (2, 1, 3)
x = sorted(zip(b, a))
list(x)

[(1, 'Charles'), (2, 'John'), (3, 'Mike')]

## 2 | Test how the core of the function will work with one channel

In [6]:
test_file = h5py.File(in_path, 'r')

# Get trial keys from one of the channels:
a = list(test_file['Channel B'].keys())
print(a)

# Transform to integer:
b = [int(x) for x in a]
print(b)

# If trial keys don't start at 1, force them to (useful for "Output" channels):
c = [(x-(min(b))+1) for x in b]
print(c)

# Sort the integers:
d = sorted(c)
print(d)

# Use numpy argsort() to get the indices that would sort the array:
e = list(np.argsort(c, axis=None))
print(e)

# Turn integers into indices (i.e. starting from 0)
f = [(x-b[0]) for x in b]
print(f)

# Get a list of sorted trial keys
# Note this is not useful as you need the indices to access the keys in a sorted way so you can extract the data from the right sweep.
z = [x for _, x in sorted(zip(c, a))]
print(z)

corrected_test_file = defaultdict(list)
for i in range(len(e)):
    correctedTrialKey = a[e[i]]
    corrected_test_file['Channel B'].append(np.array(test_file['Channel B'][correctedTrialKey]))
print(corrected_test_file['Channel B'][0:10])

uncorrected_test_file = defaultdict(list)
for i in range(len(f)):
    uncorrectedTrialKey = a[f[i]]
    uncorrected_test_file['Channel B'].append(np.array(test_file['Channel B'][uncorrectedTrialKey]))
print(uncorrected_test_file['Channel B'][0:10])

['1', '10', '100', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '4', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '5', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '6', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '7', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '8', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '9', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99']
[1, 10, 100, 11, 12, 13, 14, 15, 16, 17, 18, 19, 2, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 3, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 4, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 5, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 6, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 7, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 8, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 9, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]
[1, 10, 100, 1

In [7]:
#print(in_path)
test_file = h5py.File(in_path, 'r')
print(list(test_file.keys())) # Check channel names in file

trialKeysInHDF5 = list(test_file['Output B'].keys())
print(trialKeysInHDF5) # Check trial names in a channel

trialKeysInHDF5_int = [int(x) for x in trialKeysInHDF5]
print(trialKeysInHDF5_int)

trialKeysInHDF5_int.sort()
print(trialKeysInHDF5_int)

['Channel A', 'Channel B', 'Output A', 'Output B', 'Sweeps_Analysis', 'Time']
['11795', '11796', '11797', '11798', '11799', '11800', '11801', '11802', '11803', '11804', '11805', '11806', '11807', '11808', '11809', '11810', '11811', '11812', '11813', '11814', '11815', '11816', '11817', '11818', '11819', '11820', '11821', '11822', '11823', '11824', '11825', '11826', '11827', '11828', '11829', '11830', '11831', '11832', '11833', '11834', '11835', '11836', '11837', '11838', '11839', '11840', '11841', '11842', '11843', '11844', '11845', '11846', '11847', '11848', '11849', '11850', '11851', '11852', '11853', '11854', '11855', '11856', '11857', '11858', '11859', '11860', '11861', '11862', '11863', '11864', '11865', '11866', '11867', '11868', '11869', '11870', '11871', '11872', '11873', '11874', '11875', '11876', '11877', '11878', '11879', '11880', '11881', '11882', '11883', '11884', '11885', '11886', '11887', '11888', '11889', '11890', '11891', '11892', '11893', '11894']
[11795, 11796, 11797,

In [9]:
print(range(len(list(test_file['Time'].keys())))) # check items in time
print(test_file['Output B'].keys()) # check trial names
test_file['Output B']['11795'] # check data in a sweep

range(0, 1)
<KeysViewHDF5 ['11795', '11796', '11797', '11798', '11799', '11800', '11801', '11802', '11803', '11804', '11805', '11806', '11807', '11808', '11809', '11810', '11811', '11812', '11813', '11814', '11815', '11816', '11817', '11818', '11819', '11820', '11821', '11822', '11823', '11824', '11825', '11826', '11827', '11828', '11829', '11830', '11831', '11832', '11833', '11834', '11835', '11836', '11837', '11838', '11839', '11840', '11841', '11842', '11843', '11844', '11845', '11846', '11847', '11848', '11849', '11850', '11851', '11852', '11853', '11854', '11855', '11856', '11857', '11858', '11859', '11860', '11861', '11862', '11863', '11864', '11865', '11866', '11867', '11868', '11869', '11870', '11871', '11872', '11873', '11874', '11875', '11876', '11877', '11878', '11879', '11880', '11881', '11882', '11883', '11884', '11885', '11886', '11887', '11888', '11889', '11890', '11891', '11892', '11893', '11894']>


<HDF5 dataset "11795": shape (2500,), type "<f8">

In [10]:
test_file = h5py.File(in_path, 'r')
a = list(test_file['Channel B'].keys())
a_curated = list(test_file['Sweeps_Analysis'].keys())
print(a)
print(a_curated)

b = [int(x) for x in a]
b_curated = [int(x) for x in a_curated]
print(b)
print(b_curated)

c = [(x-(min(b))+1) for x in b]
c_curated = [(x-(min(b_curated))+1) for x in b_curated]
print(c)
print(c_curated)

e = list(np.argsort(c))
e_curated = list(np.argsort(c_curated))
print(e)
print(e_curated)

f = list(np.argsort(b))
f_curated = list(np.argsort(b_curated))
print(f)
print(f_curated)


# Test data extraction using the corrected indices
dataDict = defaultdict(list)
extractedChannels = []
for i in range(len(f_curated)):
    corrected_a_curated = a_curated[f_curated[i]]
    dataDict['Sweeps_Analysis'].append(np.array(test_file['Sweeps_Analysis'][corrected_a_curated]))
print(list(dataDict['Sweeps_Analysis']))
extractedChannels.append(dataDict['Sweeps_Analysis'])
print(extractedChannels)

['1', '10', '100', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '4', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '5', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '6', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '7', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '8', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '9', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99']
['13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48']
[1, 10, 100, 11, 12, 13, 14, 15, 16, 17, 18, 19, 2, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 3, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 4, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 5, 50, 51, 5

In [11]:
test_file = h5py.File(in_path, 'r')

# Get keys from hdf5
trialKeysInHDF5 = list(test_file['Output B'].keys())
trialKeysInHDF5_curated = list(test_file['Sweeps_Analysis'].keys())
print(trialKeysInHDF5)
print(trialKeysInHDF5_curated)

# Convert to integers so you can sort numerically
trialKeysInHDF5_int = [int(x) for x in trialKeysInHDF5]
trialKeysInHDF5_curated_int = [int(x) for x in trialKeysInHDF5_curated]
print(trialKeysInHDF5_int)
print(trialKeysInHDF5_curated_int)

# Fix those cases when trial names start at a random number (e.g. sometimes "Output" start at "14197", whereas "Channel" always start at "1")
trialKeysInHDF5_int_from1 = [(x-min(trialKeysInHDF5_int)+1) for x in trialKeysInHDF5_int] # aka numerical order
print(trialKeysInHDF5_int_from1)

trialKeysInHDF5_sorting_indices = list(np.argsort(trialKeysInHDF5_int_from1)) # aka sortIndices
print(trialKeysInHDF5_sorting_indices)

trialKeysInHDF5_sorting_indices_2 = list(np.argsort(trialKeysInHDF5_int)) # aka sortIndices
trialKeysInHDF5_curated_sorting_indices_2 = list(np.argsort(trialKeysInHDF5_curated_int)) # aka sortIndices
print(trialKeysInHDF5_sorting_indices_2)
print(trialKeysInHDF5_curated_sorting_indices_2)

trialKeysInHDF5_sorting_indices == trialKeysInHDF5_sorting_indices_2


# corrected_test_file = defaultdict(list)
# for i in range(len(trialKeysInHDF5_sorting_indices)):
#      correctedTrialKey = trialKeysInHDF5[trialKeysInHDF5_sorting_indices[i]]
#      corrected_test_file['Channel B'].append(np.array(test_file['Channel B'][correctedTrialKey]))

['11795', '11796', '11797', '11798', '11799', '11800', '11801', '11802', '11803', '11804', '11805', '11806', '11807', '11808', '11809', '11810', '11811', '11812', '11813', '11814', '11815', '11816', '11817', '11818', '11819', '11820', '11821', '11822', '11823', '11824', '11825', '11826', '11827', '11828', '11829', '11830', '11831', '11832', '11833', '11834', '11835', '11836', '11837', '11838', '11839', '11840', '11841', '11842', '11843', '11844', '11845', '11846', '11847', '11848', '11849', '11850', '11851', '11852', '11853', '11854', '11855', '11856', '11857', '11858', '11859', '11860', '11861', '11862', '11863', '11864', '11865', '11866', '11867', '11868', '11869', '11870', '11871', '11872', '11873', '11874', '11875', '11876', '11877', '11878', '11879', '11880', '11881', '11882', '11883', '11884', '11885', '11886', '11887', '11888', '11889', '11890', '11891', '11892', '11893', '11894']
['13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '2

True

In [12]:
# For the curated version of an Output:
test_file = h5py.File(in_path, 'r')

# Get keys from hdf5
trialKeysInHDF5 = list(test_file['Output B'].keys())
trialKeysInHDF5_curated = list(test_file['Sweeps_Analysis'].keys())
print(trialKeysInHDF5)
print(trialKeysInHDF5_curated)

# Convert to integers so you can sort numerically
trialKeysInHDF5_int = [int(x) for x in trialKeysInHDF5]
trialKeysInHDF5_curated_int = [int(x) for x in trialKeysInHDF5_curated]
print(trialKeysInHDF5_int)
print(trialKeysInHDF5_curated_int)

# Fix those cases when trial names start at a random number (e.g. sometimes "Output" start at "14197", whereas "Channel" always start at "1")
trialKeysInHDF5_int_from1 = [(x-min(trialKeysInHDF5_int)+1) for x in trialKeysInHDF5_int] # aka numerical order
print(trialKeysInHDF5_int_from1)

# trialKeysInHDF5_sorting_indices = list(np.argsort(trialKeysInHDF5_int_from1)) # aka sortIndices
# print(trialKeysInHDF5_sorting_indices)

trialKeysInHDF5_sorting_indices_2 = list(np.argsort(trialKeysInHDF5_int)) # aka sortIndices
trialKeysInHDF5_curated_sorting_indices_2 = list(np.argsort(trialKeysInHDF5_curated_int)) # aka sortIndices
print(trialKeysInHDF5_sorting_indices_2)
print(trialKeysInHDF5_curated_sorting_indices_2)

a = sorted(trialKeysInHDF5_curated_int)
b = trialKeysInHDF5_int_from1
print(a)
print(b)
print(b.index(a[0]))
c = [b.index(a[i]) for i in range(len(a))]
print(c)
test_file['Output B'][trialKeysInHDF5[c[0]]]

['11795', '11796', '11797', '11798', '11799', '11800', '11801', '11802', '11803', '11804', '11805', '11806', '11807', '11808', '11809', '11810', '11811', '11812', '11813', '11814', '11815', '11816', '11817', '11818', '11819', '11820', '11821', '11822', '11823', '11824', '11825', '11826', '11827', '11828', '11829', '11830', '11831', '11832', '11833', '11834', '11835', '11836', '11837', '11838', '11839', '11840', '11841', '11842', '11843', '11844', '11845', '11846', '11847', '11848', '11849', '11850', '11851', '11852', '11853', '11854', '11855', '11856', '11857', '11858', '11859', '11860', '11861', '11862', '11863', '11864', '11865', '11866', '11867', '11868', '11869', '11870', '11871', '11872', '11873', '11874', '11875', '11876', '11877', '11878', '11879', '11880', '11881', '11882', '11883', '11884', '11885', '11886', '11887', '11888', '11889', '11890', '11891', '11892', '11893', '11894']
['13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '2

<HDF5 dataset "11807": shape (2500,), type "<f8">

In [13]:
# Check the output channel loop
trialKeysInHDF5 = list(test_file['Output B'].keys())
print(trialKeysInHDF5)
trialKeysInHDF5_int = [int(x) for x in trialKeysInHDF5]
print(trialKeysInHDF5_int)
trialKeysInHDF5_sorting_indices = list(np.argsort(trialKeysInHDF5_int))
print(trialKeysInHDF5_sorting_indices)
# Transform them so they start from 1 and can be compared to the curated_channel keys:
trialKeysInHDF5_int_from1 = [(x-min(trialKeysInHDF5_int)+1) for x in trialKeysInHDF5_int]
print(trialKeysInHDF5_int_from1)

trialKeysInHDF5_curated = list(test_file['Sweeps_Analysis'].keys())
print(trialKeysInHDF5_curated)
trialKeysInHDF5_curated_int = [int(x) for x in trialKeysInHDF5_curated]
print(trialKeysInHDF5_curated_int)
trialKeysInHDF5_curated_sorting_indices = list(np.argsort(trialKeysInHDF5_curated_int))
print(trialKeysInHDF5_curated_sorting_indices)

# Sort the curated integer keys so you can use them in the list.index() step
trialKeysInHDF5_curated_int_sorted = sorted(trialKeysInHDF5_curated_int)
print(trialKeysInHDF5_curated_int_sorted)

# For the sorted curated trial keys, find the index of the value matching each curated_channel trial.
# Use this as the sorting indices
trialKeysInHDF5_sorting_indices = [trialKeysInHDF5_int_from1.index(trialKeysInHDF5_curated_int_sorted[i]) for i in range(len(trialKeysInHDF5_curated_int_sorted))]
print(trialKeysInHDF5_sorting_indices)

['11795', '11796', '11797', '11798', '11799', '11800', '11801', '11802', '11803', '11804', '11805', '11806', '11807', '11808', '11809', '11810', '11811', '11812', '11813', '11814', '11815', '11816', '11817', '11818', '11819', '11820', '11821', '11822', '11823', '11824', '11825', '11826', '11827', '11828', '11829', '11830', '11831', '11832', '11833', '11834', '11835', '11836', '11837', '11838', '11839', '11840', '11841', '11842', '11843', '11844', '11845', '11846', '11847', '11848', '11849', '11850', '11851', '11852', '11853', '11854', '11855', '11856', '11857', '11858', '11859', '11860', '11861', '11862', '11863', '11864', '11865', '11866', '11867', '11868', '11869', '11870', '11871', '11872', '11873', '11874', '11875', '11876', '11877', '11878', '11879', '11880', '11881', '11882', '11883', '11884', '11885', '11886', '11887', '11888', '11889', '11890', '11891', '11892', '11893', '11894']
[11795, 11796, 11797, 11798, 11799, 11800, 11801, 11802, 11803, 11804, 11805, 11806, 11807, 11808, 

### 2.1 | Test data extraction

In [15]:
dataDict['Sweeps_Analysis']

[array([-20.31004237, -20.63203576, -18.70007544, ..., -15.80213493,
        -14.51416137, -15.15814815]),
 array([ -3.88837926, -10.97223404, -12.26020762, ...,  -2.27841225,
         -0.34645182,  -2.60040565]),
 array([-0.66844522, -0.99043863, -5.49834627, ...,  1.26351522,
         3.19547566, -0.02445841]),
 array([15.43122544, 10.92331757,  8.34737025, ..., 10.92331757,
         7.38139001,  8.99135708]),
 array([14.14325175,  8.02537684,  7.70338342, ...,  4.16145589,
         4.80544271,  3.51746907]),
 array([ 1.26351522,  2.87348226,  1.58550862, ...,  6.41540977,
         2.22949544, -1.31243203]),
 array([ 2.87348226,  2.87348226,  5.77142295, ..., 18.00717281,
        14.78723859, 15.10923201]),
 array([11.88929782, 13.82125833, 19.93913336, ..., 26.37900193,
        24.12504792, 21.22710706]),
 array([ 26.37900193,  33.14086406,  34.10684438, ..., -23.20798284,
        -24.81794976, -25.13994314]),
 array([-23.52997622, -21.9200093 , -25.7839299 , ...,  -2.27841225,
    

In [16]:
# Define empty dictionary to populate with correctly sorted data:
data_dict = defaultdict(list)
# Define empty dictionary to populate with corrected trialKeys:
key_dict = defaultdict(list)
print(len(trialKeysInHDF5_sorting_indices))
for i in range(len(trialKeysInHDF5_sorting_indices)):
    correctedTrialKey = trialKeysInHDF5[trialKeysInHDF5_sorting_indices[i]] 
    data_dict['Output B'].append(np.array(test_file['Output B'][correctedTrialKey]))
    key_dict['Output B'].append(correctedTrialKey)
    print(data_dict['Output B'])
    print(key_dict['Output B'])

print(len(trialKeysInHDF5_curated))
print(len(key_dict['Output B']))
extracted_channels = []
corrected_trial_keys = []

# Keep only the useful channels and their trial keys:
extracted_channels.append(data_dict['Output B'])
print(len(extracted_channels[0]))
corrected_trial_keys.append(key_dict['Output B'])
print(len(corrected_trial_keys[0]))
    
# Create data frame of data:
channel_names = ['Output B']
channels_data_frame = pd.DataFrame(extracted_channels, index = channel_names, columns = corrected_trial_keys[0])
channels_data_frame

36
[array([0., 0., 0., ..., 0., 0., 0.])]
['11807']
[array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.])]
['11807', '11808']
[array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.])]
['11807', '11808', '11809']
[array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.])]
['11807', '11808', '11809', '11810']
[array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.])]
['11807', '11808', '11809', '11810', '11811']
[array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.])]
['11807', '11808', '11809', '11810', '11811', '11812'

,11807,11808,11809,11810,11811,11812,11813,11814,11815,11816,...,11833,11834,11835,11836,11837,11838,11839,11840,11841,11842
Output B,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


### 2.2 | Make a pandas data frame

In [17]:
# np.array(test_file['Sweeps_Analysis']['13'])
# channel_list = ['Channel A', 'Channel B']
# data = [np.arange(0, 18), np.arange(100, 118)]
# print(data)
# print(extractedChannels)
indices = ['Sweeps_Analysis']
df = pd.DataFrame(extractedChannels, index = indices, columns = a_curated)
df

,13,14,15,16,17,18,19,20,21,22,...,39,40,41,42,43,44,45,46,47,48
Sweeps_Analysis,"[-20.310042374699133, -20.632035761040658, -18...","[-3.8883792610270644, -10.97223403629284, -12....","[-0.6684452226398963, -0.9904386278756757, -5....","[15.431225435651891, 10.923317572881103, 8.347...","[14.143251754340366, 8.025376836201604, 7.7033...","[1.2635152152973421, 2.873482255456038, 1.5855...","[2.873482255456038, 2.873482255456038, 5.77142...","[11.889297824046958, 13.821258334791953, 19.93...","[26.379001928434015, 33.140864060409456, 34.10...","[-23.52997622421571, -21.92000930331773, -25.7...",...,"[-1.9564188417199486, -3.2443924558328088, -3....","[7.3813900092529385, 9.313350493834292, 8.3473...","[10.923317572881103, 5.127436124734345, -0.024...","[11.245310989624853, 12.211291241725363, 16.07...","[0.619528401409027, -4.532366064980151, -2.600...","[-22.242002688114866, -22.242002688114866, -18...","[-19.34406221382072, -21.276022532796944, -16....","[-16.124128324134816, -16.124128324134816, -16...","[-7.752300066135383, -7.430306667415194, -12.5...","[-7.1083132683849986, -9.362267055087196, -7.1..."
